In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import cmdstanpy
from cmdstanpy import CmdStanModel
import arviz as az
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')
import matplotlib.ticker as plt_ticker
pd.set_option('display.max_rows', 600)
from dateutil.relativedelta import relativedelta
import japanize_matplotlib
japanize_matplotlib.japanize()
import pickle

In [ ]:
hoge6 = pd.read_pickle("/home/tmiyahara/repos/Neumann-Notebook/tmiyahara/202507/data_seg.pkl")
hoge6.query("FTERM_2 >= 201909", inplace=True)
hoge6.query("FTERM_2 <= 202406", inplace=True)

consol_na_rate = (
    hoge6.groupby("CODE_SEGMENT_ADJ")["SEG_ROIC(運用ベース)"]
         .apply(lambda x: x.isna().mean())
)
valid_fsym_segment_ids = consol_na_rate[consol_na_rate < 1.0].index
hoge6.query("CODE_SEGMENT_ADJ.isin(@valid_fsym_segment_ids)", inplace=True)

consol_data = pd.read_pickle('/home/tmiyahara/repos/Neumann-Notebook/tmiyahara/202507/data_consol.pkl')
consol_data.query("FTERM_2 >= 201909", inplace=True)
consol_data.query("FTERM_2 <= 202406", inplace=True)

na_rate = (
    consol_data.groupby("FACTSET_ENTITY_ID")["ROIC(運用ベース)"]
              .apply(lambda x: x.isna().mean())
)
valid_fsym_ids = na_rate[na_rate < 1.0].index
consol_data.query("FACTSET_ENTITY_ID.isin(@valid_fsym_ids)", inplace=True)


prod_names = pd.read_pickle(
    '/home/tmiyahara/repos/Neumann-Notebook/tmiyahara/202507/prod_names_15.pkl'
)

# consol_company_product_share = pd.read_pickle(
#     '/home/tmiyahara/repos/Neumann-Notebook/tmiyahara/202507/consol_product_share.pkl'
# )
consol_company_product_share = pd.read_pickle(
    '/home/tmiyahara/repos/Neumann-Notebook/tmiyahara/202507/consol_product_share_15.pkl'
)
consol_company_product_share.query("FTERM_2 >= 201909", inplace=True)
consol_company_product_share.query("FTERM_2 <= 202406", inplace=True)

# 横方向の合計が0.4を超える行だけを残す
# consol_company_product_share = consol_company_product_share.loc[
#     consol_company_product_share[prod_names].sum(axis=1) > 0.4
# ]

company_product_share = pd.read_pickle(
    '/home/tmiyahara/repos/Neumann-Notebook/tmiyahara/202507/product_share_15.pkl'
)
company_product_share.reset_index(inplace=True)
company_product_share.query("FTERM_2 >= 201909", inplace=True)
company_product_share.query("FTERM_2 <= 202406", inplace=True)
company_product_share.reset_index(drop=True, inplace=True)

# 横方向の合計が0.4を超える行だけを残す
# company_product_share = company_product_share.loc[
#     company_product_share[prod_names].sum(axis=1) > 0.4
# ]


consol_active_fsym_ids = (
    set(consol_data["FACTSET_ENTITY_ID"].unique())
    & set(consol_company_product_share.index.get_level_values(0).unique())
)
consol_data.query("FACTSET_ENTITY_ID.isin(@consol_active_fsym_ids)", inplace=True)
consol_company_product_share.query(
    "FACTSET_ENTITY_ID.isin(@consol_active_fsym_ids)", inplace=True
)

seg_active_fsym_segment_ids = (
    set(hoge6["CODE_SEGMENT_ADJ"].unique())
    & set(company_product_share["CODE_SEGMENT_ADJ"].unique())
)
hoge6.query("CODE_SEGMENT_ADJ.isin(@seg_active_fsym_segment_ids)", inplace=True)
company_product_share.query(
    "CODE_SEGMENT_ADJ.isin(@seg_active_fsym_segment_ids)", inplace=True
)


consol_data["FTERM_2"] = pd.to_datetime(consol_data["FTERM_2"], format="%Y%m")
consol_company_product_share.index = consol_company_product_share.index.set_levels(
    pd.to_datetime(
        consol_company_product_share.index.levels[1].astype(str), format="%Y%m"
    ),
    level="FTERM_2",
)

company_product_share["FTERM_2"] = pd.to_datetime(
    company_product_share["FTERM_2"], format="%Y%m"
)


zeros1 = company_product_share[prod_names].columns[
    (company_product_share[prod_names].sum(axis=0) == 0)
]
zeros2 = consol_company_product_share[prod_names].columns[
    (consol_company_product_share[prod_names].sum(axis=0) == 0)
]
zero_rbics = list(set([*zeros1, *zeros2]))

prod_names = [name for name in prod_names if name not in zero_rbics]
consol_company_product_share.drop(columns=zero_rbics, inplace=True)
company_product_share.drop(columns=zero_rbics, inplace=True)


le = LabelEncoder()
month_map = {
    1: 3,
    2: 3,
    3: 3,
    4: 6,
    5: 6,
    6: 6,
    7: 9,
    8: 9,
    9: 9,
    10: 12,
    11: 12,
    12: 12,
}


hoge7 = hoge6.reset_index(drop=True)
hoge7["FTERM_2"] = pd.to_datetime(hoge7["FTERM_2"], format="%Y%m")
hoge7[["FACTSET_ENTITY_ID", "SEGMENT_ADJ"]] = hoge7["CODE_SEGMENT_ADJ"].str.extract(
    r"(.+?)_(.+)"
)

In [ ]:
hoge6.sort_values(by="SEG_ROIC(運用ベース)")

In [ ]:
company_product_share

In [ ]:
company_product_share = company_product_share.merge(hoge7, on=["CODE_SEGMENT_ADJ", "FTERM_2"], how="right")
consol_company_product_share = consol_company_product_share.merge(consol_data, on=["FACTSET_ENTITY_ID", "FTERM_2"], how="right")

In [ ]:
# ────────────────────────────── 1) 連結財務データ（エンティティ単位）──────────────────────────────
Y_consol = pd.pivot_table(
    data=consol_data,
    columns=["FTERM_2"],
    index="FACTSET_ENTITY_ID",
    values="ROIC(運用ベース)",
)
Y_consol.sort_index(inplace=True)

X2_consol = (
    consol_company_product_share[
        ["FACTSET_ENTITY_ID", "FTERM_2", *list(prod_names)]
    ]
    .set_index(["FACTSET_ENTITY_ID", "FTERM_2"])
)

# 時系列方向に欠損を前後補完
X2_consol.loc[:, list(prod_names)] = (
    X2_consol
        .groupby(by=["FACTSET_ENTITY_ID"], group_keys=False)
        .apply(lambda _df: _df[list(prod_names)].ffill().bfill())
        .to_numpy()
)
X2_consol.sort_index(inplace=True)

# X2_consol に存在する月だけを Y_consol の列として残す
Y_consol = Y_consol.loc[
   X2_consol.xs(X2_consol.index.get_level_values(1).unique()[0].strftime("%Y-%m-%d"), level=1).index
]


# ────────────────────────────── 2) セグメント単位データ ──────────────────────────────
Y = pd.pivot_table(
    data=hoge7,
    columns=["FTERM_2"],
    index=["CODE_SEGMENT_ADJ"],
    values="SEG_ROIC(運用ベース)",
)
Y.sort_index(inplace=True)

X2 = (
    company_product_share[
        ["CODE_SEGMENT_ADJ", "FTERM_2", *list(prod_names)]
    ]
    .set_index(["CODE_SEGMENT_ADJ", "FTERM_2"])
)

X2.loc[:, list(prod_names)] = (
    X2
        .groupby(by=["CODE_SEGMENT_ADJ"], group_keys=False)
        .apply(lambda _df: _df[list(prod_names)].ffill().bfill())
        .to_numpy()
)
X2.sort_index(inplace=True)

# X2 に存在する月だけを Y の列として残す
Y = Y.loc[
    X2.xs(X2.index.get_level_values(1).unique()[0], level=1).index
]

# X2.dropna(inplace=True)

In [ ]:
# to_vectorした後(連結)
nonna_consol = np.argwhere(~np.isnan(Y_consol.to_numpy().ravel(order='F'))).ravel(order='F') + 1 # 時間ごとに連結情報を辿る
Shared_consol = np.stack(X2_consol.groupby(level=['FTERM_2']).apply(lambda _df: _df.to_numpy()).to_list()) # 時間ごとに，セグメント×シェアの行列をつくる

nonna = np.argwhere(~np.isnan(Y.to_numpy().ravel(order='F'))).ravel(order='F') + 1 # 時間ごとにセグメント情報を辿る
Shared = np.stack(X2.groupby(level=['FTERM_2']).apply(lambda _df: _df.to_numpy()).to_list()) # 時間ごとに，セグメント×シェアの行列をつくる

In [ ]:
X2

In [ ]:
# ────────────────────────────── 0)   セグメント ID をカテゴリ整数に変換 ──────────────────────────────
seg_id = pd.pivot_table(
    data=hoge7,
    columns=["FTERM_2"],
    index=["CODE_SEGMENT_ADJ"],
    values="SEG_ROIC(運用ベース)",
)
seg_id = seg_id.reset_index().set_index("CODE_SEGMENT_ADJ")

# いま実際に観測されている月（X2 に含まれる最初の月）だけにそろえる
seg_id = seg_id.loc[
    X2.xs(X2.index.get_level_values(1).unique()[0], level=1).index
]
seg_id = seg_id.reset_index().set_index(['CODE_SEGMENT_ADJ'])
seg_id.sort_index(level=[0, 1], inplace=True)

codes = seg_id.index.get_level_values("CODE_SEGMENT_ADJ")

le.fit(codes)
# Stan へ渡す 0-始まりのカテゴリを，各月方向にコピー（= 行×列 の 2-D 行列）
codes_cat = np.repeat(
    le.transform(codes).astype("int32").reshape(-1, 1),
    Y.shape[1],
    axis=1,
)
# ────────────────────────────── 0ʹ) エンティティ（連結）側も同様 ──────────────────────────────
consol_id = pd.pivot_table(
    data=consol_data,
    columns=["FTERM_2"],
    index="FACTSET_ENTITY_ID",
    values="ROIC(運用ベース)",
)
consol_id.sort_index(level=[0, 1],inplace=True)
consol_id = consol_id.loc[
    X2_consol.xs(X2_consol.index.get_level_values(1).unique()[0].strftime("%Y-%m-%d"), level=1).index
]

codes_consol = consol_id.index.get_level_values("FACTSET_ENTITY_ID")
le.fit(codes_consol)
codes_cat_consol = np.repeat(
    le.transform(codes_consol).astype("int32").reshape(-1, 1),
    Y_consol.shape[1],
    axis=1,
)


In [ ]:
# ────────────────────────────── 1)   Stan に食わせる dict を構築 ──────────────────────────────
data_2 = dict()

data_2["Segment_N"], data_2["Time_N"] = Y.shape              # (セグメント数, 期間数)
data_2["Product_N"]   = X2[prod_names].shape[1]              # 製品種別数
data_2["Company_N"]   = len(codes_consol)                # 連結エンティティ数

# ---------- インプット行列 ----------
data_2["Share"]        = Shared                              # (t × seg × prod)
data_2["N_Share_index"] = data_2["Share"].shape[1]

# ---------- 目的変数（セグメント ROIC） ----------
data_2["Seg_ROIC"] = Y.to_numpy().ravel(order="F")[~np.isnan(Y.to_numpy().ravel(order="F"))]
data_2["non_na_index"] = nonna
data_2["N_obs"]         = len(data_2["Seg_ROIC"])
# ---------- 予測期間 ----------
data_2["N_pred_term"] = 4    # １年 (= 4Q) 先まで予測
data_2["segment_index"] = codes_cat.ravel(order="F")[~np.isnan(Y.to_numpy().ravel(order="F"))] + 1
data_2["segment_index_vec"] = codes_cat[:,0] + 1
data_2['N_segment_index_vec'] = len(data_2["segment_index_vec"])

data_2['Company_N_c'] = Y_consol.shape[0]
data_2['Share_consol'] = Shared_consol[:, :, :]
data_2['N_Share_consol_index'] = data_2['Share_consol'].shape[1]
data_2['Consol_ROIC'] = Y_consol.to_numpy().ravel(order="F")[~np.isnan(Y_consol.to_numpy().ravel(order="F"))]
data_2['non_na_index_consol'] = nonna_consol

data_2['N_obs_consol'] = len(data_2['Consol_ROIC'])
data_2['consol_index'] = codes_cat_consol.ravel(order="F")[~np.isnan(Y_consol.to_numpy().ravel(order="F"))] + 1
data_2['consol_index_vec'] = codes_cat_consol[:,0] + 1
data_2['N_consol_index_vec'] = len(data_2['consol_index_vec'])


In [ ]:
code = """data {
  int N_Share_consol_index;
  int N_Share_index;
  int N_segment_index_vec;
  int N_consol_index_vec;

  int Company_N;                 // 会社の数
  int Segment_N;                 // セグメント総数
  int Product_N;                 // 製品個数
  int Time_N;                    // 時点の数

  //------------------------------------------------------------------
  // セグメント ROIC 観測値
  //------------------------------------------------------------------
  int           N_obs;                          // 観測値総数
  vector[N_obs] Seg_ROIC;                       // 観測値ベクトル
  array[Time_N] matrix[N_Share_index,  Product_N] Share;   // セグメント×製品シェア（デザイン行列）
  array[N_obs] int  non_na_index;

  int                    N_pred_term;          // 予測期間
  array[N_obs]           int segment_index;    // セグメント ID（1-始まり）
  array[N_segment_index_vec] int segment_index_vec;

  //------------------------------------------------------------------
  // 連結 ROIC 観測値
  //------------------------------------------------------------------
  int Company_N_c;                              // 連結 ROIC を持つ企業数
  int N_obs_consol;                             // 観測値総数
  array[Time_N] matrix[N_Share_consol_index, Product_N] Share_consol; // 連結×製品シェア
  array[N_obs_consol] int  non_na_index_consol;
  array[N_consol_index]    int consol_index;
  array[N_consol_index_vec] int consol_index_vec;
  vector[N_obs_consol]     Consol_ROIC;        // 観測値ベクトル
}

parameters {
  //------------------------------------------------------------------
  // 製品 ROIC（時系列パラメータ）
  //------------------------------------------------------------------
  matrix<lower = -1, upper = 1>[Product_N, Time_N] Item_ROIC;

  //------------------------------------------------------------------
  // 階層分散パラメータ
  //------------------------------------------------------------------
  vector<lower = 0>[Product_N]   s_t;                 // 製品 ROIC の AR(2) 誤差
  vector<lower = 0>[Segment_N]   seg_sigma;           // セグメント ROIC 観測誤差
  matrix<lower = -3, upper = 3>[Segment_N,  Time_N] segment_private;
  vector<lower = 0>[Segment_N]   s_segment_private;

  vector<lower = 0>[Company_N_c] consol_sigma;        // 連結 ROIC 観測誤差
  matrix<lower = -3, upper = 3>[Company_N_c, Time_N]  consol_private;
  vector<lower = 0>[Company_N_c] s_consol_private;

  //------------------------------------------------------------------
  // Student-t の自由度
  //------------------------------------------------------------------
  real<lower = 1> nu_consol_roic;
  real<lower = 1> nu_seg_roic;
}

transformed parameters {
  //------------------------------------------------------------------
  // 各期の平均を除去した private 効果
  //------------------------------------------------------------------
  matrix[Segment_N,    Time_N] segment_private_eff;
  matrix[Company_N_c,  Time_N] consol_private_eff;

  for (t in 1:Time_N) {
    segment_private_eff[:, t] = segment_private[:, t] - mean(segment_private[:, t]);
    consol_private_eff[:,  t] =  consol_private[:, t] - mean(consol_private[:,  t]);
  }
}

model {
  //------------------------------------------------------------------
  // ハイパーパラメータの事前分布
  //------------------------------------------------------------------
  seg_sigma        ~ student_t(3, 0, 0.1);
  consol_sigma     ~ student_t(3, 0, 0.1);

  s_t              ~ student_t(3, 0, 0.5);
  s_segment_private~ student_t(3, 0, 0.5);
  s_consol_private ~ student_t(3, 0, 0.5);

  //------------------------------------------------------------------
  // 上位階層の初期分布
  //------------------------------------------------------------------
  Item_ROIC[:, 1]      ~ normal(0.04, 0.5);
  Item_ROIC[:, 2]      ~ normal(0.04, 0.5);

  segment_private[:, 1] ~ student_t(3, 0, 0.5);
  consol_private[:, 1]  ~ student_t(3, 0, 0.5);

  //------------------------------------------------------------------
  // 状態遷移（random-walk 風）
  //------------------------------------------------------------------
  for (t in 2:Time_N) {
    segment_private[:, t] ~ student_t(3, segment_private[:, t - 1],
                                         s_segment_private);
    consol_private[:,  t] ~ student_t(3,  consol_private[:,  t - 1],
                                         s_consol_private);
  }

  //------------------------------------------------------------------
  // 製品 ROIC: AR(2) 進化
  //------------------------------------------------------------------
  for (t in 3:Time_N) {
    Item_ROIC[:, t] ~ normal( (2 * Item_ROIC[:, t - 1]) - Item_ROIC[:, t - 2],
                              s_t );
  }

  //------------------------------------------------------------------
  // セグメント ROIC 尤度
  //------------------------------------------------------------------
  matrix[N_Share_index, Time_N] mu;
  for (t in 1:Time_N) {
    mu[, t] = Share[t] * Item_ROIC[, t]
            + segment_private_eff[segment_index_vec, t];
  }
  target += student_t_lpdf(
              Seg_ROIC | nu_seg_roic,
              to_vector(mu)[non_na_index],
              seg_sigma[segment_index]
            );

  //------------------------------------------------------------------
  // 連結 ROIC 尤度
  //------------------------------------------------------------------
  matrix[N_Share_consol_index, Time_N] mu_consol;
  for (t in 1:Time_N) {
    mu_consol[, t] = Share_consol[t] * Item_ROIC[, t]
                   + consol_private_eff[consol_index_vec, t];
  }
  target += student_t_lpdf(
              Consol_ROIC | nu_consol_roic,
              to_vector(mu_consol)[non_na_index_consol],
              consol_sigma[consol_index]
            );
}
"""

path = '/home/tmiyahara/repos/Neumann-Notebook/tmiyahara/202507/item_roic_rbics/stan/Item_ROIC_Beta.stan'
with open (path, 'w') as f:
    f.write(code)

In [ ]:
model = CmdStanModel(stan_file=path, force_compile=True)

In [ ]:
cmdstanpy.write_stan_json('/home/tmiyahara/repos/Neumann-Notebook/tmiyahara/202507/item_roic_rbics/data/data.json', data_2)

In [ ]:
## 変分推論
fit_1 = model.variational(data='/home/tmiyahara/repos/Neumann-Notebook/tmiyahara/202505/item_roic_rbics/data/data.json',
                          seed=1234, draws=2000, iter=500000,
                          output_dir='/home/tmiyahara/repos/Neumann-Notebook/tmiyahara/202507/item_roic_rbics/result',
                          show_console=True,
                          require_converged=False)